# Log Format Conversion

Format OpenTelemetry logs into a ClickHouse-compatible format.

This example uses the following python library to run\
"glassflow>=2.0.5" \
"Faker==30.3.0" \
"pandas" 

In [ ]:
%pip install "glassflow>=2.0.5" pandas Faker

In [3]:
import glassflow

In [4]:
# Fill Clickhouse credentials
clickhouse_config = {
    "username": "< clickhouse-username >",
    "password": "< clickhouse-password >",
    "addr": "< clickhouse host address >",
    "database": "< clickhouse database >",
    "table": "< clickhouse table >",
}

# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = "< your-personal-access-token >"

## Create Pipeline

In [5]:
client = glassflow.GlassFlowClient(
    personal_access_token=personal_access_token
)

Get the space named "examples" (or create one if no space is found)

In [8]:
list_spaces = client.list_spaces()

space_name = "examples"
for s in list_spaces.spaces:
    if s["name"] == space_name:
        space = glassflow.Space(
            personal_access_token=client.personal_access_token,
            id=s["id"], 
            name=s["name"]
        )
        break
else:
    space = client.create_space(name=space_name)

print(f"Space \"{space.name}\" with ID: {space.id}")

Space "examples" with ID: 4119e23c-c09f-4153-810c-6160ac8581eb


Display transformation function

In [ ]:
%pycat transform.py

Create a pipeline with Clickhouse as a sink

In [ ]:
with open("requirements.txt") as f:
    requirementsTxt = f.read()

pipeline = client.create_pipeline(
    name="log-format-conversion",
    space_id=space.id,
    sink_kind="clickhouse",
    sink_config=clickhouse_config,
    transformation_file="transform.py",
    requirements=requirementsTxt
)
print("Pipeline ID:", pipeline.id)

## Produce data and send it to your pipeline

Create an OpenTelemetry fake data generator:

In [14]:
from faker import Faker


def opentelemetry_datagen():
    fake = Faker()
    severity = fake.random_element(
        elements=["INFO", "DEBUG", "WARN", "ERROR"]
    )
    cloud_region = fake.random_element(
        elements=[
            'us-east-2', 'us-east-1', 'us-west-1', 'us-west-2', 'af-south-1', 'ap-east-1', 	
            'ap-south-2', 'ap-southeast-3', 'ap-southeast-5', 'ap-southeast-4', 'ap-south-1', 	
            'ap-northeast-3', 'ap-northeast-2', 'ap-southeast-1', 'ap-southeast-2', 'ap-northeast-1', 	
            'ca-central-1', 'ca-west-1', 'eu-central-1', 'eu-west-1', 'eu-west-2', 'eu-south-1', 	
            'eu-west-3', 'eu-south-2', 'eu-north-1', 'eu-central-2', 'il-central-1', 'me-south-1', 	
            'me-central-1', 'sa-east-1'
        ]
    )
    return {
        'Timestamp': fake.iso8601(),
        'SeverityText': severity,
        'Name': fake.word() + "_" + fake.word(),
        'Body': fake.sentence(nb_words=5),
        'Resource': {
            'service.name': fake.word() + "-service",
            'cloud.region': cloud_region
        },
        'Attributes': {
            'http.method': fake.http_method(),
            'http.status_code': fake.http_status_code(),
            'user.id': fake.random_int(min=1, max=99999),
            'user.ip': fake.ipv4(),
            'auth.method': 'oauth',
            'auth.success': fake.pybool()
        },
        'TraceId': fake.uuid4(),
        'SpanId': fake.uuid4(),
        'TraceFlags': '01'
    }

Create a glassflow source client and publish the generated data. 

The `n_events` variable controls the number of events that will be sent to the pipeline.

In [15]:
data_source = pipeline.get_source()

# Generate some data and send it to the pipeline. Store it locally to compare
n_events = 10
input_events = []
for i in range(n_events):
    event = opentelemetry_datagen()
    input_events.append(event)
    data_source.publish(event)

In [17]:
import pandas as pd

display(pd.DataFrame(input_events))

,time,severity,name,body,resource,attributes,trace_id,span_id,trace_flags
0,1986-06-21T20:13:20.701293,DEBUG,image_gas,Major system reduce.,"{'service.name': 'save-service', 'cloud.region...","{'http.method': 'PATCH', 'http.status_code': 5...",ba7841e5-b0a2-4248-9f14-bf5b02342d90,ea166960-eea6-49fb-b157-3b64319b797d,01
1,1987-02-25T15:29:49.873561,TRACE,development_floor,Friend reflect share.,"{'service.name': 'citizen-service', 'cloud.reg...","{'http.method': 'TRACE', 'http.status_code': 5...",3c94e751-a589-4f5b-9c64-2d75145d1362,26ad2ae4-9186-40a9-b1cc-adc68032db78,01
2,2012-02-10T15:32:02.974427,ERROR,approach_produce,Leg give thus other it.,"{'service.name': 'ready-service', 'cloud.regio...","{'http.method': 'DELETE', 'http.status_code': ...",574a5edc-97b2-4dad-b3dc-94c88812b8da,18581a8a-bc11-4537-943d-630ea8666933,01
3,1997-09-30T22:40:08.532439,DEBUG,financial_until,Else last point almost pass prove.,"{'service.name': 'letter-service', 'cloud.regi...","{'http.method': 'TRACE', 'http.status_code': 2...",ee05cd39-eedf-4b37-b086-b54b3e1f7586,948e8c5e-7a88-4f68-96b4-9ea7a0aa3240,01
4,2005-09-17T05:37:59.927345,ERROR,human_man,Film appear amount clear.,"{'service.name': 'experience-service', 'cloud....","{'http.method': 'CONNECT', 'http.status_code':...",5c6e07f8-79f2-4cbd-aa8a-85859af2546f,963c0579-a25b-42b8-bedf-f08d1dec6ddf,01
5,1983-06-25T10:35:15.415640,DEBUG,available_seek,Strategy forward film picture return.,"{'service.name': 'natural-service', 'cloud.reg...","{'http.method': 'CONNECT', 'http.status_code':...",f566ff3d-c2e1-4cde-b764-d882b353e8a1,d0fc5d68-e501-4875-886b-332d873f3e65,01
6,2004-04-07T22:04:12.393492,ERROR,appear_understand,Newspaper long work floor.,"{'service.name': 'instead-service', 'cloud.reg...","{'http.method': 'TRACE', 'http.status_code': 3...",0595eb0e-cbf6-44d9-a78e-177d52fd365c,2cfe0310-99c8-487c-9de0-302b80fabf92,01
7,2009-06-05T14:16:26.436148,TRACE,size_executive,Side moment all despite education.,"{'service.name': 'task-service', 'cloud.region...","{'http.method': 'POST', 'http.status_code': 34...",3a95a4e3-61ff-4db6-b03c-8fc7feb97c35,ec527877-27e5-480c-97a9-eedd0f4bcd0b,01
8,1987-06-05T10:12:07.286150,TRACE,hair_turn,Reveal town foot rest thought recent.,"{'service.name': 'life-service', 'cloud.region...","{'http.method': 'DELETE', 'http.status_code': ...",49d3efb6-a666-4750-9f4b-e9d452d9adbd,684dcff0-d6d0-4d09-94e7-1a565425ebf0,01
9,2021-09-29T14:50:08.727958,TRACE,whom_professor,Fish case control answer reduce.,"{'service.name': 'case-service', 'cloud.region...","{'http.method': 'DELETE', 'http.status_code': ...",8ec2ad2a-0127-4df9-be1e-27efda0b8160,a2651f0d-03b5-44af-955e-0f6fa7e9e3b9,01


## Check your Clickhouse table

Check your ClickHouse table and confirm that the events have reached your database in the correct format.